# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 4

Created by J.Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-CIRAD)

Adapted from C. Tranchant et F. Sabot (Training trasmiting science 2021)

Septembre 2021

# 1. Structural variation with Sniffles

Sniffles is a structural variation caller using third generation sequencing (PacBio or Oxford Nanopore).

It detects all types of SVs (10bp+) using evidence from split-read alignments, high-mismatch regions, and coverage analysis.

## Prepare data

In [22]:
# download  all clones fastq.gz
cd ~/SG-ONT-2021/DATA
# download your compressed CloneX 
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/all_clones_short.tar.gz

--2021-09-23 15:27:15--  https://itrop.ird.fr/ont-training/all_clones_short.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘all_clones_short.tar.gz’ not modified on server. Omitting download.



In [ ]:
#decompress it
cd ~/SG-ONT-2021/DATA
tar zxvf all_clones_short.tar.gz

In [23]:
# create SNIFFLES folder
mkdir -p ~/SG-ONT-2021/RESULTS/SNIFFLES/
cd  ~/SG-ONT-2021/RESULTS/SNIFFLES/

# declare your Clone
CLONE="Clone20"

# symbolic links of reference 
ln -s /home/jovyan/SG-ONT-2021/DATA/${CLONE}/reference.fasta .
REF="reference.fasta"

In [25]:
ls ~/SG-ONT-2021/DATA/all_clones_short/

Clone10.fastq.gz  Clone18.fastq.gz  Clone6.fastq.gz
Clone15.fastq.gz  Clone2.fastq.gz


# 1. Mapping and SV detection for all CLONES

In [20]:
# fonction bash 
run_sniffles () {
  CLONE=$1 # this is the first parametter of this fonction
  REF="reference.fasta"
  ONT="/home/jovyan/SG-ONT-2021/DATA/all_clones_short/${CLONE}.fastq.gz"
  ## Mapping using minimap2 : Mapping ONT reads (clone) vs a reference using minimap2 
  time minimap2 -t 4 -ax map-ont --MD  -R '@RG\tID:${CLONE}\tSM:${CLONE}' ${REF} ${ONT} > ${CLONE}.bam
  ## Sort BAM
  time samtools sort -@4 -o ${CLONE}_SORTED.bam ${CLONE}.bam
  # Obtain calls for a samples
  time sniffles -t 4 -s 2 -q 10 -l 10 -r 500 -m ${CLONE}_SORTED.bam -v ${CLONE}_SV.vcf
}

# -s/--min_support	Minimum number of reads that support a SV to be reported. Default: 10
# -l/--min_length	Minimum length of SV to be reported. Default: 30bp
# -q/--minmapping_qual	Minimum mapping quality of alignment to be taken into account. Default: 20
# -r/--min_seq_size	Discard read if non of its segment is larger then this. Default: 2kb

### Obtain calls for a samples

In [26]:
for i in {2,6,10,15,18}
    do
        cd  ~/SG-ONT-2021/RESULTS/SNIFFLES/
        echo "\n\n============ Clone${i}==============\n";
        run_sniffles Clone$i
    done
time

 ============ Clone2============== 
[M::mm_idx_gen::0.089*1.05] collected minimizers
[M::mm_idx_gen::0.125*1.82] sorted minimizers
[M::main::0.125*1.82] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.140*1.67] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.146*1.65] distinct minimizers: 165344 (91.75% are singletons); average occurrences: 1.156; average spacing: 5.336
[M::worker_pipeline::18.083*3.07] mapped 10241 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 4 -ax map-ont --MD -R @RG\tID:${CLONE}\tSM:${CLONE} reference.fasta /home/jovyan/SG-ONT-2021/DATA/all_clones_short/Clone2.fastq.gz
[M::main] Real time: 18.099 sec; CPU: 55.527 sec; Peak RSS: 0.414 GB

real	0m18.141s
user	0m54.392s
sys	0m1.162s
[bam_sort_core] merging from 0 files and 4 in-memory blocks...

real	0m3.259s
user	0m9.124s
sys	0m0.725s
Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 50
	Min score ratio

### Count the number of variations

In [27]:
for i in {2,6,10,15,18}
    do
        cd  ~/SG-ONT-2021/RESULTS/SNIFFLES
        echo "Clone${i}";
        grep -v "#" Clone${i}_SV.vcf | wc -l
    done

Clone2
39
Clone6
43
Clone10
67
Clone15
296
Clone18
217


# 2. Merge all the vcf files across all samples 


Check the sniffles website https://github.com/fritzsedlazeck/Sniffles/ an its wiki for more details

In [28]:
# Put all file paths over all vcf files together
ls *SV.vcf > vcf_raw_calls.txt

# We call SURVIVOR to merge these into one vcf file
conda activate survivor
cd ~/SG-ONT-2021/RESULTS/SNIFFLES
time SURVIVOR merge vcf_raw_calls.txt 1000 1 1 -1 -1 -1 merged_SURVIVOR_1kDist.vcf
conda deactivate

(survivor) (survivor) merging entries: 67
merging entries: 296
merging entries: 217
merging entries: 39
merging entries: 43

real	0m0.185s
user	0m0.113s
sys	0m0.009s
(survivor) (base) 

: 1

This will generate one VCF file for all the samples, but we lack the information 

If a SV identified in one sample but not in the other is really absent.

# 3. Force call all the SVs across all the samples

Next we ran Sniffles again across all the samples similar to this :

In [29]:
cd  ~/SG-ONT-2021/RESULTS/SNIFFLES/
for i in {2,6,10,15,18}
    do
        echo -e "\n========== REMapping Clone$i======== \n";
        sniffles -t 8 -s 2 -q 10 -l 10 -r 500 -m Clone${i}_SORTED.bam -v Clone${i}_SV.gt.vcf --Ivcf merged_SURVIVOR_1kDist.vcf
    done


(base) 
========== REMapping Clone2======== 

Automatically enabling genotype mode
Force calling SVs
Estimating parameter...
	Max dist between aln events: 5
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.0299483
	Avg INS ratio: 0.0254998
Construct Tree...
		583 SVs found in input.
	Invalid types found skipping 0 entries.
Start parsing: Chr Reference
Segmentation fault

========== REMapping Clone6======== 

Automatically enabling genotype mode
Force calling SVs
Estimating parameter...
	Max dist between aln events: 4
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.0301874
	Avg INS ratio: 0.0481301
Construct Tree...
		583 SVs found in input.
	Invalid types found skipping 0 entries.
Start parsing: Chr Reference
Segmentation fault

========== REMapping Clone10======== 

Automatically enabling genotype mode
Force calling SVs
Estimating parameter...
	Max dist between aln events: 4
	Max diff in window: 50
	Min score ratio: 2
	Avg DEL ratio: 0.0531733
	Avg INS ratio

: 1

In [ ]:
#Put all file paths over all new vcf files together
cd  ~/SG-ONT-2021/RESULTS/SNIFFLES/
ls *SV.gt.vcf > vcf_gt_calls.txt

# relauch survivor to merge vcf again and finally obtain now a fully genotyped multisample vcf
conda activate survivor
SURVIVOR merge vcf_gt_calls.txt 1000 1 1 -1 -1 -1 merged_gt_SURVIVOR_1kDist.vcf
conda deactivate

# The -1 for the minimum SV caller is necessary to obtain all calls even if they might be 0/0 in all samples.

# Have a look on the VCF file

In [ ]:
head -n 100 merged_gt_SURVIVOR_1kDist.vcf | tail -n 5

Now, you can plotting proportion of SV in all sample for VCF for example